In [1]:
import pandas as pd
import numpy as np

In [2]:
cwa_path = '../../data/labeled/2018_-_Cities_Water_Actions.csv'
cwr_path = '../../data/labeled/2018_-_Cities_Water_Risks.csv'

In [3]:
cwa = pd.read_csv(cwa_path)
cwa.head()

,Account No,Organisation,City,Country,Region,Access,C40,Reporting Year,Risks to city's water supply,Adaptation action,Action description,City Location
0,1093,City of Atlanta,Atlanta,United States of America,North America,public,NaN,2018,Increased water stress or scarcity,Investment in existing water supply infrastruc...,"The city is building a new reservoir, fixing t...","(33.749, -84.388)"
1,1184,City of Austin,Austin,United States of America,North America,public,C40,2018,Increased water stress or scarcity,Conservation awareness and education,NaN,"(30.2672, -97.7431)"
2,1184,City of Austin,Austin,United States of America,North America,public,C40,2018,Increased water stress or scarcity,Conservation incentives,NaN,"(30.2672, -97.7431)"
3,1184,City of Austin,Austin,United States of America,North America,public,C40,2018,Increased water stress or scarcity,Water use restrictions,NaN,"(30.2672, -97.7431)"
4,1184,City of Austin,Austin,United States of America,North America,public,C40,2018,Increased water stress or scarcity,Efficiency regulations or standards,NaN,"(30.2672, -97.7431)"


In [4]:
cwr = pd.read_csv(cwr_path)
cwr.head()

,Account No,Organisation,City,Country,Region,Access,C40,Reporting year,Risks to city's water supply,Timescale,Magnitude,Risk description,Current population,Population year,City Location
0,1093,City of Atlanta,Atlanta,United States of America,North America,public,NaN,2018,Increased water stress or scarcity,Short-term,Serious,The city depends on one river for water supply...,450182.0,2017,"(33.749, -84.388)"
1,1093,City of Atlanta,Atlanta,United States of America,North America,public,NaN,2018,Higher water prices,Current,Serious,The water cost burden is felt across all ZIP c...,450182.0,2017,"(33.749, -84.388)"
2,1184,City of Austin,Austin,United States of America,North America,public,C40,2018,Increased water stress or scarcity,Medium-term,Extremely serious,NaN,1265974.0,2017,"(30.2672, -97.7431)"
3,1499,Ajuntament de Barcelona,Barcelona,Spain,Europe,public,C40,2018,Increased water stress or scarcity,Current,Extremely serious,The downscaling of the climate projections at ...,1604555.0,2015,"(41.3823, 2.1775)"
4,1499,Ajuntament de Barcelona,Barcelona,Spain,Europe,public,C40,2018,Declining water quality,Medium-term,Serious,A reduction in water resources availability c...,1604555.0,2015,"(41.3823, 2.1775)"


In [5]:
# Convert columns names to lowercase
cwa.columns = cwa.columns.str.lower().str.replace(" ", "_").str.replace("'","")
cwr.columns = cwr.columns.str.lower().str.replace(" ", "_").str.replace("'","")

In [6]:
cwr.isnull().sum()

account_no                       0
organisation                     0
city                             0
country                          0
region                           0
access                           0
c40                            504
reporting_year                   0
risks_to_citys_water_supply     40
timescale                       11
magnitude                      159
risk_description               107
current_population               0
population_year                  0
city_location                    0
dtype: int64

In [7]:
cwa.isnull().sum()

account_no                       0
organisation                     0
city                             0
country                          0
region                           0
access                           0
c40                            536
reporting_year                   0
risks_to_citys_water_supply     75
adaptation_action               19
action_description             146
city_location                    0
dtype: int64

In [8]:
id_columns = list(set(cwr.columns) & set(cwa.columns))
id_columns

['reporting_year',
 'region',
 'organisation',
 'account_no',
 'risks_to_citys_water_supply',
 'city',
 'city_location',
 'access',
 'country',
 'c40']

In [9]:
cwr[id_columns].isnull().sum()

reporting_year                   0
region                           0
organisation                     0
account_no                       0
risks_to_citys_water_supply     40
city                             0
city_location                    0
access                           0
country                          0
c40                            504
dtype: int64

In [10]:
cwa[id_columns].isnull().sum()

reporting_year                   0
region                           0
organisation                     0
account_no                       0
risks_to_citys_water_supply     75
city                             0
city_location                    0
access                           0
country                          0
c40                            536
dtype: int64

In [11]:
cwr['c40'] = cwr['c40'] == 'C40'
cwr[id_columns] = cwr[id_columns].fillna('nan')

In [12]:
cwa['c40'] = cwa['c40'] == 'C40'
cwa[id_columns] = cwa[id_columns].fillna('nan')

In [13]:
len(cwa[id_columns]), len(cwa[id_columns].drop_duplicates())


(685, 484)

In [14]:
len(cwr[id_columns]), len(cwr[id_columns].drop_duplicates())

(608, 578)

In [15]:
cwa_agg = cwa.groupby(id_columns).agg(set).reset_index()
cwr_agg = cwr.groupby(id_columns).agg(set).reset_index()

In [16]:
cwr_agg

,reporting_year,region,organisation,account_no,risks_to_citys_water_supply,city,city_location,access,country,c40,timescale,magnitude,risk_description,current_population,population_year
0,2018,Africa,Abuja Federal Capital Territory,36043,Higher water prices,Abuja,"(9.07647, 7.39857)",public,Nigeria,False,{Short-term},{nan},{nan},{2440000.0},{2017}
1,2018,Africa,Abuja Federal Capital Territory,36043,Inadequate or ageing infrastructure,Abuja,"(9.07647, 7.39857)",public,Nigeria,False,{Medium-term},{nan},{nan},{2440000.0},{2017}
2,2018,Africa,Abuja Federal Capital Territory,36043,Increased water stress or scarcity,Abuja,"(9.07647, 7.39857)",public,Nigeria,False,{Short-term},{nan},{nan},{2440000.0},{2017}
3,2018,Africa,Addis Ababa City Administration,31146,Inadequate or ageing infrastructure,Addis Ababa,"(9.02887, 38.7544)",public,Ethiopia,True,{Short-term},{Serious},{nan},{3103374.0},{2013}
4,2018,Africa,Addis Ababa City Administration,31146,Increased water demand,Addis Ababa,"(9.02887, 38.7544)",public,Ethiopia,True,{Short-term},{Serious},{nan},{3103374.0},{2013}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,2018,Southeast Asia and Oceania,Wellington City Council,43937,Inadequate or ageing infrastructure,Wellington,"(-41.2865, 174.776)",public,New Zealand,False,{Current},{nan},{Wellington has a significant problem with agi...,{209102.0},{2017}
574,2018,Southeast Asia and Oceania,Wellington City Council,43937,Increased water stress or scarcity,Wellington,"(-41.2865, 174.776)",public,New Zealand,False,{Medium-term},{nan},{Wellington City suffers from a dual risk when...,{209102.0},{2017}
575,2018,Southeast Asia and Oceania,Wellington City Council,43937,nan,Wellington,"(-41.2865, 174.776)",public,New Zealand,False,{Current},{nan},{Wellington is a coastal city. Though it has n...,{209102.0},{2017}
576,2018,Southeast Asia and Oceania,Wollongong City Council,54253,Increased water stress or scarcity,Wollongong,"(-34.4278, 150.893)",public,Australia,False,{Medium-term},{nan},{Reduced water supply and increased cost of pr...,{211201.0},{2017}


In [17]:
cw_data = pd.merge(cwa_agg, cwr_agg, on=id_columns, how='outer').applymap(lambda x: {np.nan} if pd.isna(x) else x)

In [18]:
cw_data.head()

,reporting_year,region,organisation,account_no,risks_to_citys_water_supply,city,city_location,access,country,c40,adaptation_action,action_description,timescale,magnitude,risk_description,current_population,population_year
0,2018,Africa,Abuja Federal Capital Territory,36043,Higher water prices,Abuja,"(9.07647, 7.39857)",public,Nigeria,False,{Water metering},{nan},{Short-term},{nan},{nan},{2440000.0},{2017}
1,2018,Africa,Abuja Federal Capital Territory,36043,Inadequate or aging infrastructure,Abuja,"(9.07647, 7.39857)",public,Nigeria,False,{Investment in existing water supply infrastru...,{nan},{nan},{nan},{nan},{nan},{nan}
2,2018,Africa,Abuja Federal Capital Territory,36043,Increased water stress or scarcity,Abuja,"(9.07647, 7.39857)",public,Nigeria,False,{Efficiency regulations or standards},{nan},{Short-term},{nan},{nan},{2440000.0},{2017}
3,2018,Africa,City of Cape Town,35858,Increased water stress or scarcity,Cape Town,"(-33.9253, 18.4239)",public,South Africa,True,{Diversifying water supply (including new sour...,{Extensive pressure management has successfull...,{Current},{Extremely serious},{The City has generally been able to successfu...,{4174510.0},{2017}
4,2018,Africa,City of Dar es Salaam,35893,Inadequate or aging infrastructure,Dar es Salaam,"(-6.79235, 39.2083)",public,United Republic of Tanzania,True,{Investment in existing water supply infrastru...,{City provide some % of the total money correc...,{nan},{nan},{nan},{nan},{nan}


In [19]:
cw_data['city_location'] = cw_data['city_location'].apply(eval)
cw_data['latitude'] = cw_data['city_location'].apply(lambda x: x[0])
cw_data['longitude'] = cw_data['city_location'].apply(lambda x: x[1])
cw_data.drop(columns=['city_location'])
id_columns = [x for x in id_columns if x !='city_location'] + ['latitude','longitude']


In [20]:
value_columns = [ x for x in cw_data.columns if x not in id_columns]
len(cw_data[id_columns].drop_duplicates()) # seems that all

677

In [21]:
c40_data = cw_data[cw_data['c40']==True].copy()

In [22]:
# Values that can be converted back to strings from sets
singular_cols = c40_data.set_index(id_columns).applymap(lambda x: len(x)==1).product() > 0
singular_cols

city_location         False
adaptation_action     False
action_description    False
timescale             False
magnitude              True
risk_description      False
current_population     True
population_year        True
dtype: bool

In [23]:
# magnitude is nicely singular per id
c40_data.loc[:, singular_cols[singular_cols].index] = c40_data.loc[:, singular_cols[singular_cols].index].applymap(lambda x: list(x)[0])

In [24]:
c40_data = c40_data.applymap(lambda x: [t for t in x if not pd.isnull(t)] if isinstance(x,set) else x)
num_nulls = c40_data[value_columns].isnull()
num_nulls.mean()

city_location         0.000000
adaptation_action     0.000000
action_description    0.000000
timescale             0.000000
magnitude             0.339286
risk_description      0.000000
current_population    0.151786
population_year       0.151786
dtype: float64

In [25]:
list_columns = c40_data.iloc[0].apply(lambda x: isinstance(x, list))
list_columns = list_columns[list_columns].index
list_columns

Index(['adaptation_action', 'action_description', 'timescale',
       'risk_description'],
      dtype='object')

In [26]:
list_nulls = c40_data[list_columns].applymap(lambda x: len(x) == 0)
list_nulls.mean()

adaptation_action     0.348214
action_description    0.419643
timescale             0.169643
risk_description      0.321429
dtype: float64

In [27]:
list_nulls.shape, num_nulls.shape

((112, 4), (112, 8))

In [28]:
total_nulls = pd.concat([c40_data[id_columns], list_nulls, num_nulls],axis=1)


In [29]:
c40_data[total_nulls['magnitude']]

,reporting_year,region,organisation,account_no,risks_to_citys_water_supply,city,city_location,access,country,c40,adaptation_action,action_description,timescale,magnitude,risk_description,current_population,population_year,latitude,longitude
4,2018,Africa,City of Dar es Salaam,35893,Inadequate or aging infrastructure,Dar es Salaam,"(-6.79235, 39.2083)",public,United Republic of Tanzania,True,[Investment in existing water supply infrastru...,[City provide some % of the total money correc...,[],NaN,[],NaN,NaN,-6.79235,39.20830
5,2018,Africa,City of Durban,35863,Inadequate or aging infrastructure,Durban,"(-29.8587, 31.0218)",public,South Africa,True,[Investment in existing water supply infrastru...,[improvements to infrastructure are being done...,[],NaN,[],NaN,NaN,-29.85870,31.02180
9,2018,Africa,City of Johannesburg,31115,nan,Johannesburg,"(-26.2041, 28.0473)",public,South Africa,True,[Water use restrictions],[A by-law that effects water restrictions is i...,[],NaN,[],NaN,NaN,-26.20410,28.04730
15,2018,Africa,City of Lagos,31167,Inadequate or aging infrastructure,Lagos,"(6.52437, 3.3792)",public,Nigeria,True,[Investment in existing water supply infrastru...,[The State Government is currently re-engineer...,[],NaN,[],NaN,NaN,6.52437,3.37920
58,2018,East Asia,Dalian Municipal People's Government,54298,Higher water prices,Dalian,"(38.9389, 121.567)",public,China,True,"[Conservation awareness and education, Municip...",[],[],NaN,[],0.0,0.0,38.93890,121.56700
59,2018,East Asia,Dalian Municipal People's Government,54298,Increased water stress or scarcity,Dalian,"(38.9389, 121.567)",public,China,True,"[Conservation awareness and education, Municip...",[],[],NaN,[],0.0,0.0,38.93890,121.56700
60,2018,East Asia,Government of Hong Kong Special Administrative...,31169,Inadequate or aging infrastructure,Hong Kong,"(22.2881, 114.14)",public,"China, Hong Kong Special Administrative Region",True,[Other: Other],[Rehabilitation and Replacement (R&R) Programm...,[],NaN,[],NaN,NaN,22.28810,114.14000
70,2018,East Asia,Tokyo Metropolitan Government,31111,Inadequate or aging infrastructure,Tokyo,"(35.41, 139.41)",public,Japan,True,[],[Renewal of water purification plant],[],NaN,[],NaN,NaN,35.41000,139.41000
74,2018,Europe,Ajuntament de Barcelona,1499,Inadequate or aging infrastructure,Barcelona,"(41.3823, 2.1775)",public,Spain,True,[Investment in existing water supply infrastru...,[],[],NaN,[],NaN,NaN,41.38230,2.17750
89,2018,Europe,City of Oslo,14088,nan,Oslo,"(59.9139, 10.7522)",public,Norway,True,[Diversifying water supply (including new sour...,[High activity on re-opening urban rivers and ...,[],NaN,[],NaN,NaN,59.91390,10.75220


In [30]:
c40_data['magnitude'].unique()

array(['Extremely serious', nan, 'Serious', 'Less Serious'], dtype=object)

In [31]:
c40_risks = c40_data[id_columns+['magnitude']].copy()
c40_risks['low'] = c40_risks['magnitude'] == 'Less Serious'
c40_risks['medium'] = c40_risks['magnitude'] == 'Serious'
c40_risks['high'] = c40_risks['magnitude'] == 'Extremely Serious'
c40_risks['unknown'] = c40_risks['magnitude'].isna()
c40_risks = c40_risks.drop(columns=['magnitude']).groupby([x for x in id_columns if x!="risks_to_citys_water_supply"]).agg(sum).reset_index()
c40_risks

,reporting_year,region,organisation,account_no,city,access,country,c40,latitude,longitude,low,medium,high,unknown
0,2018,Africa,Addis Ababa City Administration,31146,Addis Ababa,public,Ethiopia,True,9.02887,38.75440,0,3,0,0
1,2018,Africa,City of Cape Town,35858,Cape Town,public,South Africa,True,-33.92530,18.42390,0,0,0,0
2,2018,Africa,City of Dar es Salaam,35893,Dar es Salaam,public,United Republic of Tanzania,True,-6.79235,39.20830,0,2,0,1
3,2018,Africa,City of Durban,35863,Durban,public,South Africa,True,-29.85870,31.02180,0,0,0,2
4,2018,Africa,City of Johannesburg,31115,Johannesburg,public,South Africa,True,-26.20410,28.04730,0,0,0,1
5,2018,Africa,City of Lagos,31167,Lagos,public,Nigeria,True,6.52437,3.37920,0,5,0,1
6,2018,Africa,City of Nairobi,35913,Nairobi,public,Kenya,True,-1.29206,36.82190,0,2,0,0
7,2018,East Asia,Changwon City,31186,Changwon,public,Republic of Korea,True,35.15420,126.94900,0,1,0,0
8,2018,East Asia,City of Yokohama,31113,Yokohama,public,Japan,True,35.44370,139.63800,0,1,0,0
9,2018,East Asia,Dalian Municipal People's Government,54298,Dalian,public,China,True,38.93890,121.56700,0,0,0,2


In [32]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
  
warnings.filterwarnings(action = 'ignore')
  
import gensim
from gensim.models import Word2Vec

In [33]:
import spacy
try:
    nlp = spacy.load("en_core_web_md")
except:
    spacy.cli.download('en_core_web_md')
    nlp = spacy.load("en_core_web_md")

In [47]:
c40_data_action_description = c40_data.action_description.apply(lambda x:nlp(' '.join(x)).vector if x else np.nan)

In [48]:
processed = nlp(' '.join(c40_data.action_description.iloc[0]))

In [57]:
replace = c40_data_action_description.mean()
c40_data_action_description = c40_data_action_description.apply(lambda x: replace if not isinstance(x,np.ndarray) else x)

In [50]:
c40_data_action_description.mean()

array([-1.69838220e-02,  1.12242132e-01, -4.27668765e-02, -1.07277714e-01,
       -2.24944055e-02,  1.82229653e-03, -4.84297201e-02,  9.69366636e-03,
       -8.00681487e-02,  1.96918797e+00, -2.88623124e-01, -3.12868110e-03,
        5.24468906e-02, -2.05828492e-02, -1.43529683e-01, -3.26475501e-02,
       -3.96126546e-02,  1.39790189e+00, -1.78410500e-01, -5.49613424e-02,
        2.95284968e-02,  2.96106488e-02, -8.09554830e-02, -3.81493680e-02,
        6.10170215e-02,  2.53590359e-03,  8.97452061e-04,  3.86917852e-02,
        1.62690580e-02,  3.74630955e-03, -2.12879516e-02, -1.05412006e-02,
       -2.45062970e-02, -3.26531976e-02,  6.56235889e-02, -9.41012129e-02,
        2.57159155e-02, -3.78166395e-03,  5.20633534e-02, -5.21873683e-02,
       -1.16431266e-02,  5.76804094e-02,  4.38490584e-02, -6.50522858e-02,
       -6.95356801e-02,  1.57016478e-02, -1.24762565e-01,  5.84449209e-02,
       -4.59467769e-02, -2.17325846e-03,  6.76942244e-02,  7.07759261e-02,
        3.39993578e-03,  